In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
data_cols = ['user id','movie id','rating','timestamp']
item_cols = ['movie id','movie title','release date','video release date','IMDb URL','unknown','Action','Adventure','Animation','Childrens','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance ','Sci-Fi','Thriller','War' ,'Western']
user_cols = ['user id','age','gender','occupation','zip code']

In [4]:
users = pd.read_csv('ml-100k/u.user', sep='|', names=user_cols, encoding='latin-1')
item = pd.read_csv('ml-100k/u.item', sep='|', names=item_cols, encoding='latin-1')
data = pd.read_csv('ml-100k/u.data', sep='\t', names=data_cols, encoding='latin-1')

# Usaremos o arquivo u.data primeiro, pois ele contém ID do usuário, IDs do filme e classificações. Esses três elementos são tudo o que precisamos para determinar a similaridade dos usuários com base em suas classificações para um filme específico. 

# Primeiro, vamos classificar o DataFrame pela ID do usuário e, em seguida, dividiremos o conjunto de dados em um conjunto de treinamento e um conjunto de testes (só precisamos de um usuário para o treinamento).

In [5]:
utrain = (data.sort_values('user id'))[:99832]
print(utrain.tail())

       user id  movie id  rating  timestamp
73676      942       479       4  891283118
67222      942       604       4  891283139
95675      942       478       5  891283017
85822      942       659       5  891283161
68192      942       487       4  891282985


In [6]:
utest = (data.sort_values('user id'))[99833:]
print(utest.head())

       user id  movie id  rating  timestamp
91841      943       132       3  888639093
91810      943       204       3  888639117
77956      943        94       4  888639929
87415      943        53       3  888640067
77609      943       124       3  875501995


# Vamos converter para um NumPy Array para facilitar a iteração.

In [10]:
utrain = utrain[['user id', 'movie id', 'rating']].values
print(utrain)

[[  1  55   5]
 [  1 203   4]
 [  1 183   5]
 ...
 [942 478   5]
 [942 659   5]
 [942 487   4]]


In [11]:
utest = utest[['user id', 'movie id', 'rating']].values
print(utest)

[[ 943  132    3]
 [ 943  204    3]
 [ 943   94    4]
 [ 943   53    3]
 [ 943  124    3]
 [ 943  672    5]
 [ 943 1047    2]
 [ 943  541    4]
 [ 943  226    4]
 [ 943  449    1]
 [ 943  470    4]
 [ 943   11    4]
 [ 943  526    4]
 [ 943  549    1]
 [ 943   28    4]
 [ 943  184    5]
 [ 943   22    4]
 [ 943 1188    3]
 [ 943  196    5]
 [ 943  233    5]
 [ 943   41    4]
 [ 943  796    3]
 [ 943   56    5]
 [ 943  231    2]
 [ 943    9    3]
 [ 943   42    5]
 [ 943  205    5]
 [ 943  443    2]
 [ 943  585    1]
 [ 943   73    3]
 [ 943   54    4]
 [ 943  765    3]
 [ 943   58    4]
 [ 943  122    1]
 [ 943   50    4]
 [ 943   72    2]
 [ 943  391    2]
 [ 943  181    4]
 [ 943  234    3]
 [ 943  385    4]
 [ 943   76    4]
 [ 943  274    3]
 [ 943  188    4]
 [ 943  468    2]
 [ 943 1028    2]
 [ 943  415    1]
 [ 943  227    1]
 [ 943   68    4]
 [ 943  367    4]
 [ 943   98    5]
 [ 943  546    4]
 [ 943  186    5]
 [ 943  943    5]
 [ 943  412    2]
 [ 943   55    5]
 [ 943  20

# Crie um users_list, que é uma lista de usuários que contém uma lista de filmes classificados por ele.

In [12]:
users_list = []
for i in range(1,943):
    list = []
    for j in range(0,len(utrain)):
        if utrain[j][0] == i:
            list.append(utrain[j])    
        else:
            break
    utrain = utrain[j:]
    users_list.append(list) 
    
print(len(users_list))

942


# Defina uma função chamada EucledianScore. O objetivo do EucledianScore é medir a similaridade entre dois usuários com base em suas avaliações dadas aos filmes que eles têm em comum. Mas e se os usuários tiverem apenas um filme em comum? Ter mais filmes em comum é um grande sinal de semelhança. Portanto, se os usuários tiverem menos de quatro filmes em comum, atribuiremos a eles um EucledianScore alto.

In [13]:
def EucledianScore(train_user, test_user):
    sum = 0
    count = 0
    for i in test_user:
        score = 0
        for j in train_user:
            if(int(i[1]) == int(j[1])):
                score= ((float(i[2])-float(j[2]))*(float(i[2])-float(j[2])))
                count= count + 1        
            sum = sum + score
    if(count<4):
        sum = 1000000           
    return(math.sqrt(sum))

# Agora vamos iterar sobre users_list e encontrar a similaridade dos usuários com o test_user por meio dessa função e anexar o EucledianScore juntamente com o User ID em uma lista separada score_list. 
# Em seguida, vamos converter a lista em DataFrame, e assim classificá-lo pelo EucledianScore e, finalmente, convertê-lo em um score_matrix NumPy Array para facilitar a iteração.

In [16]:
score_list = []               
for i in range(0,942):
    score_list.append([i+1,EucledianScore(users_list[i], utest)])
score = pd.DataFrame(score_list, columns = ['user id','Eucledian Score'])
score = score.sort_values(by = 'Eucledian Score')
print(score)
score_matrix = score.values

     user id  Eucledian Score
309      310         1.732051
138      139         3.872983
45        46         4.000000
208      209         4.242641
557      558         4.582576
..       ...              ...
110      111      1000.000000
719      720      1000.000000
106      107      1000.000000
104      105      1000.000000
694      695      1000.000000

[942 rows x 2 columns]


# Agora vemos que o usuário com ID 310 tem a menor pontuação de distância euclidiana e, portanto, a maior similaridade. 

# Então, agora precisamos obter a lista de filmes que não são comuns entre os dois usuários. Faça duas listas. Veja a lista completa dos filmes que estão lá em USER_ID 310. E então a lista de filmes comuns. Converta essas listas em conjuntos e obtenha a lista de filmes a serem recomendados.

In [17]:
user= int(score_matrix[0][0])
common_list = []
full_list = []
for i in utest:
    for j in users_list[user-1]:
        if(int(i[1])== int(j[1])):
            common_list.append(int(j[1]))
        full_list.append(j[1])

common_list = set(common_list)  
full_list = set(full_list)
recommendation = full_list.difference(common_list)

# Agora precisamos criar uma lista compilada dos filmes junto com suas classificações médias. Mesclar os arquivos "item" e "dados". Em seguida, agrupe pelos títulos de filmes, selecione as colunas que você precisa e, em seguida, encontre as avaliações médias de cada filme. Em seguida, expresse o Dataframe como um Array NumPy.

In [32]:
item_list = (((pd.merge(item,data).sort_values(by = 'movie id')).groupby('movie title')))[['movie id', 'movie title', 'rating']]
item_list = item_list.mean()
item_list['movie title'] = item_list.index
item_list = item_list.values

# Agora encontramos os filmes em item_list por IDs da recomendação. Em seguida, anexe-os a uma lista separada.
# Imprima-os e suas recomendações estão prontas!

In [55]:
recommendation_list = []
for i in recommendation:
    recommendation_list.append(item_list[i-1]))
    
recommendation = (pd.DataFrame(recommendation_list,columns = ['movie id','mean rating' ,'movie title'])).sort_values(by = 'mean rating', ascending = False)
print(recommendation[['mean rating','movie title']])

TypeError: unsupported operand type(s) for -: 'str' and 'int'